In [2]:
import random 
import numpy as np

import plotly.express as px 
import plotly.io as pio
import plotly.graph_objects as go

from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP

In [9]:
# Make Up a Dataset 

from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.1)
#For y: 0-->-1 and 1-->1
y = 2*y-1

# Plot the Dataset
fig = px.scatter(x=X[:,0], y=X[:,1], color=y)

fig.update_layout(
    title="Dataset",
    xaxis_title="X1",
    yaxis_title="X2",
    legend_title="Class",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig.show()


In [12]:
# Define the Model
model = MLP(2, [16,16,1]) # 2 inputs, 3 hidden layers with 16 neurons each, 1 output
print("Number of parameters:", len(model.parameters()))

Number of parameters: 337


In [76]:
# Define the Loss Function
def loss_fn(batch_size = None):
    if batch_size is None:
        batch_size = len(X)
    idx = np.random.choice(len(X), batch_size, replace=False)
    X_batch = X[idx]
    y_batch = y[idx]

    
    #Convert the inputs to Value objects
    inputs = [list(map(Value,xRow)) for xRow in X_batch]

    #Forward Pass With Inputs
    outputs = [model(x) for x in inputs]

    #Compute Loss
    #Max-Margin Loss Function

    #Output is either 1 or -1
    #y is either 1 or -1
    #If y and output are the same, then loss is 0, else loss is 1 so we use max(0, 1-y*output)
    loss = 0
    size = 0 
    for yi, output in zip(y_batch, outputs):
        loss += (1-yi*output).relu()
        size += 1
    loss /= size

    #Also get Accuracy
    correct = 0 
    for yi, output in zip(y_batch, outputs):
        if yi*output.data > 0:
            correct += 1
    accuracy = correct/size

    return loss, accuracy




In [99]:

for k in range(1):
    loss, accuracy = loss_fn()

    #Backpropagate
    model.zero_grad()
    loss.backward()

    #Move in the opposite direction of the gradient
    learning_rate = 1 - .9*k/100
    for p in model.parameters():
        p.data -= learning_rate*p.grad

    print(f"Step {k}, Loss: {loss.data:.3f}, Accuracy: {accuracy*100:.2f}%")

Step 0, Loss: 0.011, Accuracy: 99.80%


In [81]:
model.layers[0].neurons[0].w

[Value(data=0.12272881695752402, grad=-0.006014424044499598),
 Value(data=-0.38194770898975694, grad=0.033742188277726015)]

In [107]:
# Plot the Decision Boundary Using Plotly

# Create a meshgrid
x1 = np.linspace(-2, 3, 100)
x2 = np.linspace(-2, 3, 100)
xx1, xx2 = np.meshgrid(x1, x2)

xmesh = np.array([xx1.ravel(), xx2.ravel()]).T

# Convert the inputs to Value objects
inputs = [list(map(Value,xRow)) for xRow in xmesh]

#Forward Pass With Inputs
outputs = [model(x) for x in inputs]

#Convert the outputs to numpy array
outputs = np.array([output.data for output in outputs])

#Reshape the outputs
outputs = outputs.reshape(xx1.shape)


array([[  4.78244149,   5.00910564,   5.24085955, ...,  21.6633026 ,
         21.84314474,  22.02720957],
       [  4.41554869,   4.64221284,   4.868877  , ...,  21.28000261,
         21.46306251,  21.64808426],
       [  4.04865589,   4.27532004,   4.5019842 , ...,  20.89891545,
         21.0839372 ,  21.26895894],
       ...,
       [-12.52621367, -12.35849556, -12.19077745, ...,  -2.78802683,
         -2.64719104,  -2.50635524],
       [-12.63782641, -12.4701083 , -12.29287656, ...,  -2.90822998,
         -2.76739418,  -2.62655839],
       [-12.74943915, -12.58172104, -12.38301413, ...,  -3.02843313,
         -2.88759733,  -2.74676153]])

In [108]:

#Plot the decision boundary
fig = go.Figure(data=[go.Contour(x=x1, y=x2, z=outputs, colorscale='RdBu', reversescale=True, opacity=0.9)])

#Plot the dataset
fig.add_trace(go.Scatter(x=X[:,0], y=X[:,1], mode='markers', marker=dict(color=y, colorscale='RdBu', reversescale=True), showlegend=False))

fig.update_layout(
    title="Decision Boundary",
    xaxis_title="X1",
    yaxis_title="X2",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)

fig.show()